# Connected Component Analysis

## Objects

### Pixel Neighborhoods

#### Jumps
1. 1-jump is only allowed along the column, or the row. Diagonal jumps are not allowed.
2. 2-jump: in a sequence of jumps, one may only jump in row and column direction once.

In [ ]:
import imageio.v3 as iio
import ipympl
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski

%matplotlib widget

In [ ]:
def connected_components(filename, sigma=1.0, t = 0.5, connectivity=2):
    image = iio.imread(filename)
    gray_image = ski.color.rgb2gray(image)
    blurred_image = ski.filters.gaussian(gray_image, sigma)
    # 아래 기준도 애매함. 이미지에 따라서 달라짐. 아래 샘플에서는 임계점이 되는 배경색은 다른 값보다 높은 값을 가지므로 < 를 사용함.
    # 배경색이 검은색에 가깝다면 > 를 사용하는 것이 맞다.
    binary_mask = blurred_image < t
    labeled_image, count = ski.measure.label(
        binary_mask, connectivity=connectivity, return_num=True)
    return labeled_image, count

In [ ]:
# Blur의 강도를 높일 수록 정확도는 높아질 수 있으나 정확한 값을 찾는 것은 매우 어렵다.
labeled_image, count = connected_components(
    filename="data/shapes-01.jpg",
    sigma=1,
    t=0.9,
    connectivity=1
)
print(f"Found {count} shapes.")
fig, ax = plt.subplots()
ax.imshow(labeled_image)
ax.set_axis_off();

In [ ]:
colored_label_image = ski.color.label2rgb(labeled_image, bg_label=0)

fig, ax = plt.subplots()
ax.imshow(colored_label_image)
ax.set_axis_off()

## Morphometrics - Describe object features with numbers

In [ ]:
object_features = ski.measure.regionprops(labeled_image)
object_areas = [feature["area"] for feature in object_features]
object_areas

In [ ]:
histogram, bin_edges = np.histogram(object_areas)
fig, ax = plt.subplots()
ax.plot(bin_edges[0:-1], histogram)
ax.hist(object_areas)

In [ ]:
regionprop1 = object_features[3]
print("label",  regionprop1.label)



In [ ]:
def enhanced_connected_components(
        filename, sigma=1.0, t = 0.5, connectivity=2, min_area=0):
    image = iio.imread(filename)
    gray_image = ski.color.rgb2gray(image)
    blurred_image = ski.filters.gaussian(gray_image, sigma)
    # 아래 기준도 애매함. 이미지에 따라서 달라짐. 아래 샘플에서는 임계점이 되는 배경색은 다른 값보다 높은 값을 가지므로 < 를 사용함.
    # 배경색이 검은색에 가깝다면 > 를 사용하는 것이 맞다.
    binary_mask = blurred_image < t
    object_mask = ski.morphology.remove_small_objects(binary_mask, min_size=min_area)
    labeled_image, count = ski.measure.label(object_mask, connectivity=connectivity, return_num=True)
    return labeled_image, count

In [ ]:
# Blur의 강도를 높일 수록 정확도는 높아질 수 있으나 정확한 값을 찾는 것은 매우 어렵다.
labeled_image_2, count = enhanced_connected_components(
    filename="data/shapes-01.jpg",
    sigma=1,
    t=0.9,
    connectivity=1,
    min_area=10000
)
print(f"Found {count} shapes.")

In [ ]:
colored_label_image = ski.color.label2rgb(labeled_image_2, bg_label=0)

fig, ax = plt.subplots()
ax.imshow(colored_label_image)
ax.set_axis_off()

print("Found", count, "objects in the image")

In [ ]:
object_areas = np.array([feature["area"] for feature in ski.measure.regionprops(labeled_image_2)])
object_areas = np.insert(0, obj=1, values = object_areas)
print(object_areas)
colored_area_image = object_areas[labeled_image_2]

fig, ax = plt.subplots()
im = ax.imshow(colored_area_image)
cbar = fig.colorbar(im, ax=ax, shrink=0.85)
cbar.ax.set_title("Area")
ax.set_axis_off()

In [ ]:
labeled_image